# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [104]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
import numpy as np

In [ ]:
# !pip install biopython pandas

# Load Data 

In [223]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### 모든 행이 WT인 컬럼 제거 (train, test 모두)

In [224]:
# 두 데이터프레임에서 모든 값이 'WT'인 컬럼을 각각 찾음
columns_all_wt_train = train.columns[(train == 'WT').all(axis=0)]
columns_all_wt_test = test.columns[(test == 'WT').all(axis=0)]

# train과 test에서 모든 값이 'WT'인 컬럼의 합집합 찾기
union_columns = columns_all_wt_train.union(columns_all_wt_test)

# 합집합에 해당하는 컬럼을 train과 test에서 삭제
train = train.drop(columns=union_columns)
test = test.drop(columns=union_columns)

# 결과 확인
# print("모든 값이 'WT'인 컬럼을 삭제한 train 데이터프레임:")
# print(train.head())
# print("모든 값이 'WT'인 컬럼을 삭제한 test 데이터프레임:")
# print(test.head())


### 표기 규칙 수정 (78s)

In [225]:
import pandas as pd
import re

# 돌연변이 표기 수정 규칙 적용 함수
def process_mutations(mutation_string):
    if pd.isnull(mutation_string):
        return None  # 결측치 처리

    # 여러 개의 변이를 ',' 또는 ' ' 로 나누어 처리
    mutations = mutation_string.split(',')
    processed_mutations = []

    for mutation in mutations:
        mutation = mutation.strip()
        
        # 다중 변이 패턴: '711_712FL>FL' -> 'F711F L712L'로 수정
        multi_match = re.match(r'^(\d+_\d+)([A-Z]+)>([A-Z*]+)$', mutation)
        if multi_match:
            positions = multi_match.group(1)  # '197_198'
            from_aa_seq = multi_match.group(2)  # 'YQ'
            to_aa_seq = multi_match.group(3)    # '**' 또는 특정 아미노산 코드

            pos_list = positions.split('_')
            new_mutation = []
            for i, (from_aa, to_aa) in enumerate(zip(from_aa_seq, to_aa_seq)):
                # '*'와 같은 특수한 문자가 들어오면 그대로 유지
                if to_aa == '*':
                    new_mutation.append(f"{from_aa}{pos_list[i]}*")
                else:
                    new_mutation.append(f"{from_aa}{pos_list[i]}{to_aa}")
            processed_mutations.append(' '.join(new_mutation))

        # '*' 돌연변이 패턴은 그대로 추가
        elif '*' in mutation:
            processed_mutations.append(mutation)
        
        # 기본적인 패턴: 그대로 추가
        else:
            processed_mutations.append(mutation)

    # 중복 제거 후 반환
    unique_mutations = list(dict.fromkeys(processed_mutations))  # 중복 제거
    return ', '.join(unique_mutations)

In [226]:
train_processed = train.applymap(process_mutations)

C:\Users\sukja\AppData\Local\Temp\ipykernel_8176\1431569657.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_processed = train.applymap(process_mutations)


In [227]:
test_processed = test.applymap(process_mutations)

C:\Users\sukja\AppData\Local\Temp\ipykernel_8176\4212444730.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_processed = test.applymap(process_mutations)


### misense driver 적용 (1m 26s)

In [228]:
misense_driver = pd.read_csv('missense_driver.csv', sep='\t')

In [229]:
misense_driver['Gene name'] = misense_driver['Gene name'].str.split('_').str[0]

In [230]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [231]:
misense_driver = misense_driver[misense_driver['Gene name'].isin(yes)]

In [232]:
misense_driver_dict = misense_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [233]:
import pandas as pd

# 다중 돌연변이를 처리하는 함수
def update_misense_driver(train, misense_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_driver_dict[gene]
            
            # 해당 열에 대해 처리
            def process_mutations(mutation_string):
                if pd.isnull(mutation_string):  # 결측치 처리
                    return mutation_string

                # 돌연변이 문자열을 공백으로 분리하여 각 변이를 개별적으로 처리
                mutations = mutation_string.split()
                
                # 각 변이가 mutation_list에 있는지 확인하여 30으로 교체
                processed_mutations = [str(30) if mut in mutation_list else mut for mut in mutations]
                
                # 변이를 다시 공백으로 연결하여 반환
                return ' '.join(processed_mutations)
            
            # apply 함수를 사용하여 각 셀에 대해 처리
            train[gene] = train[gene].apply(process_mutations)
    
    return train


In [234]:
# 함수 적용
train_processed2 = update_misense_driver(train_processed, misense_driver_dict)
test_processed2 = update_misense_driver(test_processed, misense_driver_dict)



In [235]:
train_processed2[train_processed2['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1758,TRAIN_1758,STES,15,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,15,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,15,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,15,WT,WT,WT,WT,WT,WT,WT,...,A1913D,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,15,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,15,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [236]:
test_processed2[test_processed2['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,15,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### missense passenger 적용 (2m 44s)

In [237]:
misense_passenger = pd.read_csv('missense_passenger.csv', sep='\t')

In [238]:
misense_passenger['Gene name'] = misense_passenger['Gene name'].str.split('_').str[0]

In [239]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [240]:
misense_passenger = misense_passenger[misense_passenger['Gene name'].isin(yes)]

In [241]:
misense_passenger_dict = misense_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [242]:
def update_misense_passenger(train, misense_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_passenger_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(0) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [243]:
train_processed3=update_misense_passenger(train_processed2,misense_passenger_dict)
test_processed3=update_misense_passenger(test_processed2,misense_passenger_dict)

In [244]:
train_processed3[train_processed3['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,E291E,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2969,TRAIN_2969,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2984,TRAIN_2984,COAD,0,WT,WT,0,S2192N F1854F K376R,WT,WT,V1698V E1295* S1071L,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [245]:
test_processed3[test_processed3['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT


### silent driver 적용 (51s)

In [246]:
silent_driver = pd.read_csv('silent_driver.csv', sep='\t')

In [247]:
silent_driver['Gene name'] = silent_driver['Gene name'].str.split('_').str[0]

In [248]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [249]:
silent_driver = silent_driver[silent_driver['Gene name'].isin(yes)]

In [250]:
silent_driver_dict = silent_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [251]:
def update_silent_driver(train, silent_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_driver_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(30) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [252]:
train_processed4 = update_silent_driver(train_processed3,silent_driver_dict)
test_processed4 = update_silent_driver(test_processed3,silent_driver_dict)

In [253]:
train_processed4[train_processed4['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
594,TRAIN_0594,CESC,15,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1758,TRAIN_1758,STES,15,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,15,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,15,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,15,WT,WT,WT,WT,WT,WT,WT,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,15,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,15,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [254]:
test_processed4[test_processed4['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,15,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### silent passenger 적용 (2m 42s)

In [255]:
silent_passenger = pd.read_csv('silent_passenger.csv', sep='\t')

In [256]:
silent_passenger['Gene name'] = silent_passenger['Gene name'].str.split('_').str[0]

In [257]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [258]:
silent_passenger = silent_passenger[silent_passenger['Gene name'].isin(yes)]

In [259]:
silent_passenger_dict = silent_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [260]:
def update_silent_passenger(train, silent_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_passenger_dict[gene]
            
            # 해당 열에 대해 값이 돌연변이 리스트에 있는지 확인하고 있으면 30으로 대체
            #train[gene] = train[gene].apply(lambda x: 0 if x in mutation_list else x )
            train[gene] = train[gene].apply(lambda x: str(0) if not isinstance(x, (int, float)) and x in mutation_list else x)

    
    return train


In [261]:
train_processed5 = update_silent_passenger(train_processed4,silent_passenger_dict)
test_processed5 = update_silent_passenger(test_processed4,silent_passenger_dict)

In [262]:
train_processed5[train_processed5['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1502,TRAIN_1502,COAD,0,WT,WT,0,WT,WT,WT,V1774V T311T,...,K162T S664S A779V T1526M L1806V R2115*,WT,WT,WT,Q1572Q H730Q I688I,WT,0,WT,WT,WT
1606,TRAIN_1606,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,0,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [263]:
test_processed5[test_processed5['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1337,TEST_1337,0,WT,WT,WT,WT,WT,WT,WT,WT,...,K1345*,WT,WT,WT,WT,WT,WT,WT,WT,A430E
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT
2214,TEST_2214,0,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [264]:
train_processed5.to_csv('train_amino_ver4_pre.csv')
test_processed5.to_csv('test_amino_ver4_pre.csv')

### amino acid 함수 정의 + train,test set 적용 (1m 26s)

In [147]:
import pandas as pd
from Bio.Align import substitution_matrices

# BLOSUM62 매트릭스 가져오기
blosum62 = substitution_matrices.load('BLOSUM62')

# 20개의 표준 아미노산 리스트 정의
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G',
               'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S',
               'T', 'W', 'Y', 'V']

# 데이터 프레임 생성
blo_score = pd.DataFrame(index=amino_acids, columns=amino_acids)

# 데이터 프레임에 점수 할당
for aa1 in amino_acids:
    for aa2 in amino_acids:
        score = blosum62[aa1, aa2]
        blo_score.loc[aa1, aa2] = score

In [268]:
train = pd.read_csv('train_amino_ver4_pre.csv')

In [274]:
train['A2M'].unique()

array(['WT', 'R895R', 'E1462K', 'K289T', 'I175F', 'D533D', 'L903L', '0',
       'P799S', 'M713I', 'A615A', 'R586*', 'Q1051H D152D', 'D638N',
       'I984S', 'F352S', 'L1214L F572L', 'R1297R', '15', 'I822I F229L',
       'I481N', 'A1061V I1056I', 'S781S', 'E153K', 'T885A', 'F495L',
       'M121I', 'D897H', 'H582H', 'L311P', 'D641E', 'S63F A79A', 'S1249F',
       'E1311K E342K R147C E1311K E342K R147C', 'T411S', 'T1027T R719H',
       'V207L', 'V206V', 'C1321Y I848I P529H', 'V1255V V814V V551M',
       'L670L', 'S136*', 'I175I', 'Y1104D', 'F1389F T1239T V129V L13I',
       'A788A', 'S782C', 'A1108T', 'S202S', 'L1067I', 'Y1216Y', 'K1336K',
       'L1101L', 'S785P', 'S1210S', 'T1229S', 'G69E', 'Q1220H',
       'S1102S S150S', 'F1028V', 'M121V', 'R1122H', 'H156Y I175I', 'P36S',
       'G1094V', 'P463P', 'V120V', 'N1038T', 'F365L', 'P157P',
       '15 G280G S191Y', 'P969S E840*', 'G69R', 'L965R L461L E113*',
       'L606L', 'K625K', 'S1071S', 'G367G', 'S276F', 'I1091V', 'T1285I',
       'H44

In [148]:
blo_score = blo_score.applymap(lambda x: int(x))

C:\Users\sukja\AppData\Local\Temp\ipykernel_8176\1618710540.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blo_score = blo_score.applymap(lambda x: int(x))


In [149]:
for i in blo_score.columns:
    blo_score[i] = blo_score.loc[i,i] - blo_score[i]
blo_score = blo_score.T

In [218]:
blo_score

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,0,5,6,6,4,5,5,4,6,5,5,5,5,6,5,3,4,7,6,4
R,6,0,5,7,8,4,5,7,5,8,7,3,6,8,7,6,6,8,7,8
N,8,6,0,5,9,6,6,6,5,9,9,6,8,9,8,5,6,10,8,9
D,8,8,5,0,9,6,4,7,7,9,10,7,9,9,7,6,7,10,9,9
C,9,12,12,12,0,12,13,12,12,10,10,12,10,11,12,10,10,11,11,10
Q,6,4,5,5,8,0,3,7,5,8,7,4,5,8,6,5,6,7,6,7
E,6,5,5,3,9,3,0,7,5,8,8,4,7,8,6,5,6,8,7,7
G,6,8,6,7,9,8,8,0,8,10,10,8,9,9,8,6,8,8,9,9
H,10,8,7,9,11,8,8,10,0,11,11,9,10,9,10,9,10,10,6,11
I,5,7,7,7,5,7,7,8,7,0,2,7,3,4,7,6,5,7,5,1


In [275]:
import pandas as pd
import re
import numpy as np


# 단일 변이를 분류하는 함수
def classify_mutation(mutation):
    # 결측치 처리
    if pd.isnull(mutation):
        return None

    mutation = str(mutation).strip()

    # 특수 변이 유형 체크
    if mutation == 'WT':
        return 0  # Wild Type (돌연변이 없음)

    if 'fs' in mutation or 'del' in mutation or '*' in mutation:
        return 45  # 프레임시프트, 삭제, 중단 코돈 돌연변이

    # 단일 아미노산 변이 패턴 매칭 (예: 'R496Q', 'L1700L')
    match = re.match(r'^([A-Z])(\d+)([A-Z])$', mutation)
    if match:
        from_aa = match.group(1)  # 원래 아미노산
        position = match.group(2) # 위치 (여기서는 사용하지 않음)
        to_aa = match.group(3)    # 변이된 아미노산

        # 아미노산 코드가 유효한지 확인
        if from_aa not in blo_score.index or to_aa not in blo_score.columns:
            return None  # 알 수 없는 아미노산 코드

        # 침묵 돌연변이 체크 (아미노산이 동일한 경우)
        if from_aa == to_aa:
            return 0  # 침묵 돌연변이

        # 변이 스코어 데이터프레임에서 점수 가져오기
        score = blo_score.loc[from_aa, to_aa]

        # 점수가 결측치인 경우 처리
        if pd.isnull(score):
            return None  # 또는 기본 점수로 지정

        return int(score)  # 변이 점수 반환
    else:
        # 패턴 매칭 실패한 경우
        return None  # 매칭 실패한 경우 7 반환
# 다중 치환을 처리하는 함수 정의
def classify_multiple_mutations(mutation_string):
    # 결측치 처리
    if pd.isnull(mutation_string):
        return None  # 또는 특정 코드로 지정 가능

    # 변이 문자열을 공백으로 분리
    mutations = set(str(mutation_string).strip().split())

    labels = []
    for mutation in mutations:
        # 숫자인 경우 또는 숫자로 변환 가능한 경우 그대로 유지
        try:
            mutation = float(mutation)  # 숫자로 변환 시도
            mutation = int(mutation) if mutation.is_integer() else mutation  # 정수형이면 int, 실수형이면 그대로 유지
            labels.append(mutation)
        except ValueError:
            # 숫자가 아닌 경우 기존 로직 사용
            label = classify_mutation(mutation)
            if label is not None:
                labels.append(label)

    if labels:
        # 합계 반환
        return sum(labels)
    else:
        return None  # 또는 특정 코드로 지정 가능

In [ ]:
# TRAIN 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train_processed5.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    train_processed5[col] = train_processed5[col].apply(classify_multiple_mutations)


In [277]:
# TRAIN 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    train[col] = train[col].apply(classify_multiple_mutations)


In [185]:
train = pd.read_csv('train_amino_ver4_pre.csv')

In [ ]:
classify_multiple_mutations

In [182]:
# TEST 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in test_processed5.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    test_processed5[col] = test_processed5[col].apply(classify_multiple_mutations)


### CSV 저장 & 불러오기
- 함수 적용 시간이 오래걸리기때문에 여기까지 trian,test를 csv로 저장 후 불러내기 해서 사용
- 저장한 train파일에 컬럼(유전자) 수는 원래 유전자수(4865)에서 모든 행이 'WT'이 제거됨 (4226열)
- 저장된 파일은 전처리가 끝난 상태 -> 모델만 수정해가면서 train,test 불러와서 반복해서 돌리기


In [186]:
#4개의 csv파일함수적용 후 'PS_mapping_ver3.csv'로 저장하기 ( 최초 1회만 실행)
train_processed5.to_csv('PS_mapping_ver3.csv')
test_processed5.to_csv('test_wtcut.csv')

In [187]:
# 두번째부터 여기부터 실행하기 train,test 는 전처리가 끝났음, 바로 모델에 학습시키면 됨
import pandas as pd
train = pd.read_csv('PS_mapping_ver3.csv')
train = train.iloc[:,1:]
test = pd.read_csv('test_wtcut.csv')
test = test.iloc[:,1:]

# Data Preprocessing

In [278]:
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# Model definition

- aminoacid version 3 ( 공통 'WT' 제거 했을 때)
- Train Accuracy: 74.64%
- Test Accuracy: 36.66%

In [280]:
#amino acid version3 + basic model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# 1. 데이터 준비
# 타겟: 'SUBCLASS', 특징: 'SUBCLASS'와 'ID'를 제외한 나머지 열
X = train.drop(columns=['SUBCLASS', 'ID'])  # 특징 데이터 (SUBCLASS를 제외한 모든 열)
y = train['SUBCLASS']  # 타겟 데이터 (SUBCLASS)

# 2. 학습 세트와 테스트 세트로 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 3. XGBoost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y.unique()), random_state=42)

# 4. 모델 학습
model.fit(X_train, y_train)

# 5. 예측
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 6. 정확도 계산
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 7. 결과 출력
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Train Accuracy: 81.45%
Test Accuracy: 37.55%


Based on prior findings, driver mutations in key oncogenes tend to have a much larger impact than single amino acid substitutions like W→A unless the W→A mutation occurs at a functional "hotspot" critical to protein function. In terms of severity, if driver mutations are given a severity of 3, the tryptophan-to-alanine substitution might be closer to a 1 to 2 severity, depending on the context .

If we take frameshift mutations to be at a severity level of 3, driver mutations, though not always as immediately devastating to protein function as frameshift mutations, can still hold significant severity, particularly in genes critical for cancer growth, such as TP53. Therefore, driver mutations might be slightly less severe on average but still hold a significant impact, ranging between 2 to 2.5 in terms of severity compared to frameshift mutations​

In [282]:
blo_score

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,0,5,6,6,4,5,5,4,6,5,5,5,5,6,5,3,4,7,6,4
R,6,0,5,7,8,4,5,7,5,8,7,3,6,8,7,6,6,8,7,8
N,8,6,0,5,9,6,6,6,5,9,9,6,8,9,8,5,6,10,8,9
D,8,8,5,0,9,6,4,7,7,9,10,7,9,9,7,6,7,10,9,9
C,9,12,12,12,0,12,13,12,12,10,10,12,10,11,12,10,10,11,11,10
Q,6,4,5,5,8,0,3,7,5,8,7,4,5,8,6,5,6,7,6,7
E,6,5,5,3,9,3,0,7,5,8,8,4,7,8,6,5,6,8,7,7
G,6,8,6,7,9,8,8,0,8,10,10,8,9,9,8,6,8,8,9,9
H,10,8,7,9,11,8,8,10,0,11,11,9,10,9,10,9,10,10,6,11
I,5,7,7,7,5,7,7,8,7,0,2,7,3,4,7,6,5,7,5,1


In [215]:
# amino acid version3 + basic model with 5-fold cross-validation
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
import xgboost as xgb

# 1. 데이터 준비
# 타겟: 'SUBCLASS', 특징: 'SUBCLASS'와 'ID'를 제외한 나머지 열
X = train.drop(columns=['SUBCLASS', 'ID'])  # 특징 데이터 (SUBCLASS와 ID를 제외한 모든 열)
y = train['SUBCLASS']  # 타겟 데이터 (SUBCLASS)

# 2. XGBoost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y.unique()), random_state=42)

# 3. Stratified K-Fold 설정 (5-폴드)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. 정확도 점수를 계산하기 위한 스코어러 정의
accuracy_scorer = make_scorer(accuracy_score)

# 5. 교차 검증 수행
cv_scores = cross_val_score(model, X, y, cv=skf, scoring=accuracy_scorer)

# 6. 결과 출력
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean() * 100:.2f}%")
print(f"Std CV Accuracy: {cv_scores.std() * 100:.2f}%")


Cross-Validation Accuracy Scores: [0.35455278 0.35322581 0.36451613 0.38387097 0.3516129 ]
Mean CV Accuracy: 36.16%
Std CV Accuracy: 1.20%


In [ ]:
# best model 만들어서 저장해주세요~
model.save_model('amino3_cat_smote.cbm')

# Inference

In [193]:
# test 전처리가 끝난 상태로 저장됐기 때문에 바로 Inference 가능
test = test.drop(columns=['ID'])

In [194]:
predictions = model.predict(test)

In [195]:
original_labels = le_subclass.inverse_transform(predictions)

# Submisson

In [196]:
submisson = pd.read_csv("./sample_submission.csv")

In [197]:
submisson["SUBCLASS"] = original_labels

In [198]:
submisson.to_csv('amino3_sgb.csv', encoding='UTF-8-sig', index=False)